In [ ]:
# 
# This Notebook requires the following environment variables (.env file):
# 

# VIDEO_TO_AUDIO_LOCAL_WORKING_DIR=TODO
# VIDEO_TO_AUDIO_INPUT_CONTAINER_SAS_URI=TODO
# VIDEO_TO_AUDIO_OUTPUT_CONTAINER_SAS_URI=TODO

In [ ]:
import os
import ffmpeg
import shutil
from utils import get_container_client_from_sas, download_blobs, upload_files
from dotenv import load_dotenv

#
# Main
#

load_dotenv()

LOCAL_WORKING_DIR = os.getenv("VIDEO_TO_AUDIO_LOCAL_WORKING_DIR")
INPUT_CONTAINER_SAS_URI = os.getenv("VIDEO_TO_AUDIO_INPUT_CONTAINER_SAS_URI")
OUTPUT_CONTAINER_SAS_URI = os.getenv("VIDEO_TO_AUDIO_OUTPUT_CONTAINER_SAS_URI")

print("Starting video to audio conversion...")

input_folder = os.path.join(LOCAL_WORKING_DIR, "converted-video-input")
output_folder = os.path.join(LOCAL_WORKING_DIR, "converted-video-output")

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

input_container_client = get_container_client_from_sas(INPUT_CONTAINER_SAS_URI)
output_container_client = get_container_client_from_sas(OUTPUT_CONTAINER_SAS_URI)

print("Downloading blobs from input container...")
video_files = download_blobs(input_container_client, local_folder=input_folder, suffix=".mp4")

print(f"Convert {len(video_files)} videos to audio from {input_folder}...")

audio_files = []
for video_file in video_files:
        try:
            output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(video_file))[0]}.mp3")

            # Use FFmpeg to extract the segment
            (
                ffmpeg
                .input(video_file)
                .output(output_path, **{'q:a': 0, 'map': 'a'})
                .run(quiet=True, overwrite_output=True)
            )

            audio_files.append(output_path)
                       
        except Exception as e:
            print(f"Error creating segment {i+1}: {str(e)}")

if audio_files:    
    uploaded_files = upload_files(output_container_client, audio_files)
    print(f"Uploaded {len(uploaded_files)} audio files to output container.")

shutil.rmtree(input_folder)
shutil.rmtree(output_folder)

print("Video to audio conversion completed.")